In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

from utils.preprocessing import get_texts
from utils.preprocessing import get_texts, stop_words

In [2]:
# TODO: Modify here for different sectors and score_types
# set the sector and ESG type for the analysis
# sector = "Industrials"
sector = "Energy"
# sector = "Energy"

# score_type = "governanceScore"
score_type = "environmentScore"
# score_type = "socialScore"

In [3]:
df_esg_score = pd.read_excel("data/esg_score.xlsx", sheet_name = "data")

In [4]:
df_esg_score["sector"].dropna().unique()

array(['Consumer Cyclical', 'Energy', 'Industrials', 'Healthcare',
       'Basic Materials', 'Consumer Defensive', 'Utilities', 'Technology',
       'Financial Services', 'Communication Services', 'Real Estate'],
      dtype=object)

In [38]:
d = {"combination": [], "bad_tickers": [], "good_tickers": []}

for sec in df_esg_score["sector"].dropna().unique():
    for s_type in ["governanceScore", "environmentScore", "socialScore"]:
        tickers = df_esg_score[df_esg_score["sector"] == sec]["Company"]
        esgs = df_esg_score[df_esg_score["sector"] == sec][["Company", "socialScore", "governanceScore", "environmentScore"]]
        score = esgs[s_type]
        alpha = 0.25
        upper_score = np.quantile(score, 1 - alpha)
        lower_score = np.quantile(score, alpha)

        bad_companies = esgs[esgs[s_type] > upper_score]["Company"].values
        good_companies = esgs[esgs[s_type] < lower_score]["Company"].values

        # print(sec)
        d["combination"].append(sec + "_" + s_type)
        d["good_tickers"].append(','.join(good_companies))
        d["bad_tickers"].append(','.join(bad_companies))

ddf = pd.DataFrame(data=d)

In [42]:
ddf.to_csv("data/goodvbad/goodcbad_companies.csv")

## Get texts for companies

In [6]:
tickers = df_esg_score[df_esg_score["sector"] == sector]["Company"]
esgs = df_esg_score[df_esg_score["sector"] == sector][["Company", "socialScore", "governanceScore", "environmentScore"]]

In [7]:
len(tickers)

20

In [8]:
score = esgs[score_type]

In [46]:
alpha = 0.3
upper_score = np.quantile(score, 1 - alpha)
lower_score = np.quantile(score, alpha)

In [47]:
bad_companies = esgs[esgs[score_type] > upper_score]["Company"].values
good_companies = esgs[esgs[score_type] < lower_score]["Company"].values

In [48]:
good_companies

array(['NOV', 'OKE', 'HAL', 'SLB', 'WMB', 'KMI'], dtype=object)

In [49]:
bad_companies

array(['COG', 'MRO', 'CVX', 'EOG', 'APA', 'OXY'], dtype=object)

In [50]:
esgs[esgs[score_type] > upper_score]

,Company,socialScore,governanceScore,environmentScore
1,COG,14.01,9.28,23.39
3,MRO,10.27,8.70,23.76
4,CVX,10.67,10.21,20.29
7,EOG,11.06,8.24,19.67
8,APA,8.88,7.96,21.98
10,OXY,10.85,6.75,20.00


In [51]:
bad_companies_score = esgs[esgs[score_type] > upper_score][score_type].values
good_companies_score = esgs[esgs[score_type] < lower_score][score_type].values

In [52]:
avg_bad = np.mean(bad_companies_score)
avg_good = np.mean(good_companies_score)
print(avg_bad, avg_good)

21.515 9.834999999999999


In [53]:
print(upper_score, lower_score)

19.376 14.691


In [54]:
ticker_library = pd.read_csv(os.path.join("data", "tickers.csv"))
good_cik = []
bad_cik = []
for ticker in good_companies:    
    try:
        # for a given ticker, find its cik number through th ticker library
        good_cik.append(ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:])
    except:
        # if could not find cik, give it a empty cik
        good_cik.append('')

for ticker in bad_companies:    
    try:
        # for a given ticker, find its cik number through th ticker library
        bad_cik.append(ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:])
    except:
        # if could not find cik, give it a empty cik
        bad_cik.append('')

/Users/luckywang/Documents/Document/Course Material/Fall 2021/esg_nlp/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [55]:
ret_good = get_texts(good_cik, good_companies)

6it [00:01,  3.26it/s]


In [56]:
ret_bad = get_texts(bad_cik, bad_companies)

6it [00:02,  2.25it/s]


In [57]:
good_docs = ret_good["docs"]
bad_docs = ret_bad["docs"]

In [65]:
# TODO: Modify here for different ngram range
n_min = 2
n_max = 3
cv = CountVectorizer(max_df=0.6, stop_words=stop_words, max_features=200, ngram_range=(n_min, n_max))
word_count_vector = cv.fit_transform(good_docs + bad_docs)
feature_names = cv.get_feature_names()

/Users/luckywang/Documents/Document/Course Material/Fall 2021/esg_nlp/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['10'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [67]:
count_feature = word_count_vector.toarray().sum(axis=0)

In [68]:
feature_names = cv.get_feature_names()

In [69]:
d = {"word": [], "good_score": [], "bad_score": [], "good_score_all": []
    , "bad_score_all": [], "count": [], "good_nums": [], "bad_nums": []}

for feature_idx, word in enumerate(feature_names):
    good_sum = bad_sum = good_num = bad_num = 0

    for i, doc_set in enumerate(good_docs):
        if word in doc_set:
            good_num += 1
            good_sum += good_companies_score[i]
    for i, doc_set in enumerate(bad_docs):
        if word in doc_set:
            bad_num += 1
            bad_sum += bad_companies_score[i]
    
    # print("word: {}".format(word))
    d["word"].append(word) 
    
    if good_num:
        d["good_score"].append(good_sum / good_num)
    else:
        d["good_score"].append(0)
    if bad_num:
        d["bad_score"].append(bad_sum / bad_num)
    else:
        d["bad_score"].append(0)

    d["good_score_all"].append(good_sum / len(good_docs))
    d["bad_score_all"].append(bad_sum / len(bad_docs))

    d["count"].append(count_feature[feature_idx])
    d["good_nums"].append(good_num)
    d["bad_nums"].append(bad_num)


In [70]:
df = pd.DataFrame(data=d)


In [71]:
df.head()

,word,good_score,bad_score,good_score_all,bad_score_all,count,good_nums,bad_nums
0,accompanying note consolidated,13.830,21.886667,2.305000,10.943333,1143,1,3
1,accompanying note integral,10.545,21.705000,3.515000,14.470000,1584,2,4
2,adjusted ebitda,10.975,0.000000,7.316667,0.000000,1051,4,0
3,adopted pursuant,9.294,21.880000,7.745000,7.293333,1043,5,2
4,adopted pursuant section,9.294,21.880000,7.745000,7.293333,1037,5,2


In [248]:
df["diff"] = abs(df["good_nums"] - df["bad_nums"])

In [249]:
df = df.sort_values("diff", ascending=False)#.head(60)

In [117]:
df.head(20)

,word,good_score,bad_score,good_score_all,bad_score_all,count,good_nums,bad_nums,isGood
0,accompanying note consolidated,13.830000,21.886667,2.305000,10.943333,1143,1,3,-1
1,accompanying note integral,10.545000,21.705000,3.515000,14.470000,1584,2,4,-1
2,adjusted ebitda,10.975000,0.000000,7.316667,0.000000,1051,4,0,1
3,adopted pursuant,9.294000,21.880000,7.745000,7.293333,1043,5,2,1
4,adopted pursuant section,9.294000,21.880000,7.745000,7.293333,1037,5,2,1
5,among williams,8.980000,0.000000,1.496667,0.000000,1164,1,0,0
6,analysis continued,8.980000,0.000000,1.496667,0.000000,1269,1,0,0
7,apache corporation,8.765000,20.550000,2.921667,10.275000,3760,2,3,0
8,apache corporation subsidiary,0.000000,21.980000,0.000000,3.663333,1711,0,1,0
9,attributable common,8.765000,20.990000,2.921667,6.996667,1139,2,2,0


In [251]:
goodvbad_path = os.path.join("data", "goodvbad")
if not os.path.isdir(goodvbad_path):
    os.mkdir(goodvbad_path)

In [252]:
df.round(2).to_csv("data/goodvbad/{}_{}_{}_n{}-{}.csv".format(sector[:8], score_type[:3], alpha, n_min, n_max))

## Appendix

In [ ]:
# To remove all the previous files earlier than 2020
for cik in tqdm(bad_cik):
    tenk_path = os.path.join("data", "10k", cik, "rawtext")
    all_raws = os.listdir(tenk_path)
    for filename in all_raws:
        if filename[0] == '.':
            continue
        year = int(filename.split('_')[1].split('-')[0])
        # print(year)
        if year < 2020:
            # print(os.path.join(tenk_path, filename))
            os.remove(os.path.join(tenk_path, filename))
    
    tenq_path = os.path.join("data", "10q", cik, "rawtext")
    all_raws = os.listdir(tenq_path)
    for filename in all_raws:
        if filename[0] == '.':
            continue
        year = int(filename.split('_')[1].split('-')[0])
        # print(year)
        if year < 2020:
            # print(os.path.join(tenk_path, filename))
            os.remove(os.path.join(tenq_path, filename))


In [ ]:
# Remove all the previous incompleted scraping
dirname = os.path.join("data", "10q")
all_files = os.listdir(dirname)

import shutil
for filename in all_files:
    pkldir = os.path.join(dirname, filename, "pickle")
    if os.path.isdir(pkldir):
        if not os.path.exists(os.path.join(pkldir, "agg_texts.pkl")):
            shutil.rmtree(os.path.join(dirname, filename))

In [4]:
df = pd.read_csv("energy_good_vs_bad_uni_bi_tri.csv", index_col=0)

In [5]:
df.head()

,word,good_score,bad_score,good_score_all,bad_score_all,good_nums,bad_nums,diff
0,accompanying note consolidated,9.773333,23.7600,4.886667,3.960000,3,1,2
1,accompanying note integral,10.557500,21.6950,7.038333,7.231667,4,2,2
2,acmp,0.000000,21.9800,0.000000,3.663333,0,1,1
3,activity cash flow,10.975000,21.0660,7.316667,17.555000,4,5,1
4,adjusted ebitda,0.000000,22.2825,0.000000,14.855000,0,4,4


In [237]:
comp = pd.read_csv("data/sp500_component_stocks.csv")


In [241]:
comp[comp['A'] == "C"]["Agilent Technologies Inc."]

79    Citigroup Inc.
Name: Agilent Technologies Inc., dtype: object

In [140]:
comp[comp['A'].isin(good_companies)]["Agilent Technologies Inc."]

44                   Apache Corporation
111         Cabot Oil & Gas Corporation
128                 Chevron Corporation
163                  EOG Resources Inc.
322            Marathon Oil Corporation
358    Occidental Petroleum Corporation
Name: Agilent Technologies Inc., dtype: object

In [141]:
comp[comp['A'].isin(bad_companies)]["Agilent Technologies Inc."]

215            Halliburton Company
274      Kinder Morgan Inc Class P
343    National Oilwell Varco Inc.
354                     ONEOK Inc.
411                Schlumberger NV
487        Williams Companies Inc.
Name: Agilent Technologies Inc., dtype: object

In [9]:
ticker_library = pd.read_csv(os.path.join("data", "tickers.csv"))

/Users/luckywang/Documents/Document/Course Material/Fall 2021/esg_nlp/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [27]:
def get_cik(ticker):
    """ Get the cik for the ticker specified by the input argument 
    Input:
        ticker(str): ticker of the company e.g. "FB"
    """
    print(ticker)
    return ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:]
    

In [29]:
def get_ciks(tickers):
    ciks = []

    for ticker in tickers:
        ciks.append(get_cik(ticker))

    return ciks

In [86]:
def ngrams(s, n):
    """ Get all the n-gram for input texts s
    Input:
        s (str): A string of texts with each word separated by a whitespace
        n (int): n-gram to extract
    Return:
        [str]: A list of string in the following format ([['a', 'b'], ['b', 'c'], ['c', 'd']])
    """
    
    s = s.split(' ')
    output = []
    for i in range(len(s) - n + 1):
        output.append(s[i:i+n])

    return output

# ngrams('a b c d', 2) # [['a', 'b'], ['b', 'c'], ['c', 'd']]

In [126]:
"accompanying note consolidated" in df["word"].values

True

In [175]:
def get_count(doc, df_dict, n_min, n_max):
    """ Count the number of good and bad words occurred in the document
    Input:
        doc (str): A string with all the words in the documents
        df_dict (pd.DataFrame): A DataFrame with word and isGood column, generated by previous section
        n_min, n_max (int): specify the ngram range used to generate the dictionary, should be consistent with how df_dict is generated
    Return:
        (dict): A dictionary with value good_count and bad_count
    """
    grams = []
    for n in range(n_min, n_max + 1):
        grams.extend([' '.join(li) for li in ngrams(doc, 2)])
    
    good_count = bad_count = 0
    
    for g in grams:
        if g in df_dict["word"].values:
            val = df_dict[df_dict["word"] == g]["isGood"].values
            if val == 1:
                good_count += 1
            elif val == -1:
                bad_count += 1

    return {"good_count": good_count, "bad_count": bad_count}
    

In [174]:
def validation(df_topk, val_tickers, dict_threshold, val_true):
    """ Perform the validation step
    The validation rationale: Companies whose score are in upper 50% group are considered "bad" companies and the corresponding val_true = 1; 0 otherwise (in lower 50% group, which is considered a good company)
    Input:
        df_topk (pd.DataFrame): containes the sector specific dict
        val_tickers (list): A list of tickers to be validated
        dict_threshold (int): A gram is considered as a good word if its good_nums - bad_nums > threshold
        val_true (list): A list of true labels for each companies
    """
    diff = df_topk["good_nums"] - df_topk["bad_nums"]
    df_topk["isGood"] = diff.apply(lambda x: 1 if x > dict_threshold else (
        -1 if x < -dict_threshold else 0))

    # 1 if good_nums - bad_nums > threshold; -1 if good_nums - bad_nums < -threshold; 0 otherwise
    val_ciks = [get_cik(ticker) for ticker in val_tickers]
    
    ret_texts = get_texts(val_ciks, val_tickers) 

    val_pred = []
    for doc in tqdm(ret_texts["docs"]):
        ret = get_count(doc, df_topk[["word", "isGood"]], 2, 3)

        if ret["good_count"] - ret["bad_count"] > 0:
            val_pred.append(1)
        else:
            val_pred.append(0)
    
    print("val_pred: {}".format(val_pred))
    cm = confusion_matrix(val_true, val_pred)
    print("Confusion Matrix: \n{}".format(cm))
    return val_pred

In [163]:
bad_companies

array(['COG', 'MRO', 'CVX', 'EOG', 'APA', 'OXY'], dtype=object)

In [170]:
# tickers = ['NOV', 'OKE', 'HAL', 'SLB', 'WMB', 'KMI']
tickers = list(good_companies[:1]) + list(bad_companies[:1])
ciks = [get_cik(ticker) for ticker in tickers]

In [171]:
ret = get_texts(ciks, tickers) 

2it [00:01,  1.83it/s]


In [172]:
val_true = [1 for _ in good_companies[:1]] + [0 for _ in bad_companies[:1]]

In [173]:
v = validation(df, tickers, 2, val_true)

2it [00:01,  1.73it/s]
100%|██████████| 2/2 [01:25<00:00, 42.98s/it]

[1, 0]
[[1 0]
 [0 1]]


In [156]:
v

[1, 1]

In [107]:
dict_threshold = 1
diff = df["good_nums"] - df["bad_nums"]
df["isGood"] = diff.apply(lambda x: 1 if x > dict_threshold else (
    -1 if x < -dict_threshold else 0))

In [108]:
df.head()

,word,good_score,bad_score,good_score_all,bad_score_all,count,good_nums,bad_nums,isGood
0,accompanying note consolidated,13.830,21.886667,2.305000,10.943333,1143,1,3,-1
1,accompanying note integral,10.545,21.705000,3.515000,14.470000,1584,2,4,-1
2,adjusted ebitda,10.975,0.000000,7.316667,0.000000,1051,4,0,1
3,adopted pursuant,9.294,21.880000,7.745000,7.293333,1043,5,2,1
4,adopted pursuant section,9.294,21.880000,7.745000,7.293333,1037,5,2,1


In [109]:
r = get_count(ret["docs"][1], df[["word", "isGood"]], 2, 3)

In [110]:
r

{'good_count': 0, 'bad_count': 0}

In [116]:
df[["word", "isGood"]]

,word,isGood
0,accompanying note consolidated,-1
1,accompanying note integral,-1
2,adjusted ebitda,1
3,adopted pursuant,1
4,adopted pursuant section,1
...,...,...
195,trustee filed,0
196,unconsolidated affiliate,1
197,well incident,0
198,williams company,0


## Generate N-grams based on sectors

In [16]:
ticker_library = pd.read_csv(os.path.join("data", "tickers.csv"))

/Users/luckywang/Documents/Document/Course Material/Fall 2021/esg_nlp/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
sector = "Energy"
# sector = "Energy"

# score_type = "governanceScore"
score_type = "environmentScore"

In [22]:
# sectors = ['Consumer Cyclical', 'Energy', 'Industrials', 'Healthcare',
#        'Basic Materials', 'Consumer Defensive', 'Utilities', 'Technology',
#        'Financial Services', 'Communication Services', 'Real Estate']
sectors = ['Consumer Cyclical', 'Technology', 'Financial Services']
score_types = ["governanceScore", "environmentScore", "socialScore"]

In [46]:
not_listed = ["BBWI"]

In [61]:
with open('data/n_grams/Technology_environmentScore.pkl', 'rb') as handle:
    feature_names = pickle.load(handle)

In [62]:
feature_names

['accrued unpaid',
 'acquired intangible',
 'additional information regarding',
 'adopted pursuant',
 'adopted pursuant section',
 'affect financial result',
 'allowance credit',
 'amortized cost',
 'amount senior',
 'annual report year',
 'average price',
 'average selling',
 'average selling price',
 'benefit pension',
 'billion billion',
 'cash flow hedge',
 'channel partner',
 'class action',
 'cloud computing',
 'cloud license',
 'cloud service',
 'commercial paper',
 'company adopted',
 'company also',
 'company belief',
 'company entered',
 'company recorded',
 'compared fiscal',
 'compared million',
 'compared period',
 'compared three',
 'compared three month',
 'comprehensive loss',
 'condensed financial',
 'condensed financial statement',
 'consolidated condensed',
 'consolidated condensed financial',
 'consolidated statement income',
 'consolidated statement operation',
 'constant currency',
 'contingent consideration',
 'continuing operation',
 'contract manufacturer',
 'c

In [48]:
import pickle

In [ ]:
df_esg_score = pd.read_excel("data/esg_score.xlsx", sheet_name = "data")

In [ ]:
for sector in sectors:
    for score_type in score_types:
        esgs = df_esg_score[df_esg_score["sector"] == sector][["Company", "socialScore", "governanceScore", "environmentScore"]]
        score = esgs[score_type]
        alpha = 0.3
        upper_score = np.quantile(score, 1 - alpha)
        lower_score = np.quantile(score, alpha)

        bad_companies = list(esgs[esgs[score_type] > upper_score]["Company"].values)
        good_companies = list(esgs[esgs[score_type] < lower_score]["Company"].values)

        for t in not_listed:
            if t in bad_companies:
                bad_companies.remove(t)
            if t in good_companies:
                good_companies.remove(t)

        tickers = good_companies + bad_companies
        ciks = get_ciks(tickers)
        ret = get_texts(ciks, tickers)
        docs = ret["docs"]
        
        n_min = 2
        n_max = 3
        cv = CountVectorizer(max_df=0.7, stop_words=stop_words, max_features=200, ngram_range=(n_min, n_max))
        word_count_vector = cv.fit_transform(docs)
        feature_names = cv.get_feature_names()

        with open('data/n_grams/{}_{}.pkl'.format(sector, score_type), 'wb') as handle:
            pickle.dump(feature_names, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [52]:
with open('data/n_grams/{}_{}.pkl'.format(sector, score_type), 'wb') as handle:
    pickle.dump(feature_names, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
tickers = df_esg_score[df_esg_score["sector"] == sector]["Company"]
esgs = df_esg_score[df_esg_score["sector"] == sector][["Company", "socialScore", "governanceScore", "environmentScore"]]
ciks = get_ciks(tickers)

In [19]:
ret = get_texts(ciks, tickers)
docs = ret["docs"]

20it [00:05,  3.46it/s]


In [20]:
# TODO: Modify here for different ngram range
n_min = 2
n_max = 3
cv = CountVectorizer(max_df=0.6, stop_words=stop_words, max_features=200, ngram_range=(n_min, n_max))
word_count_vector = cv.fit_transform(docs)
feature_names = cv.get_feature_names()

/Users/luckywang/Documents/Document/Course Material/Fall 2021/esg_nlp/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['10'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
